#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [49]:
!pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable


In [50]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=18804 sha256=3afcc888d939daa36ae23c718006fe8d5a6a73adc4755dc952e3b328962bcd08
  Stored in directory: /tmp/pip-ephem-wheel-cache-dd6u3jfz/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [51]:
import os
import pandas as pd

In [52]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree, Boston311KerasNLP


## Get latest file URLS and Current Date Ranges

In [53]:
latest_URLS = Boston311LogReg.get311URLs()

In [54]:
print(latest_URLS)

{'2023': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmpur_spi78.csv', '2022': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmpfm8veglw.csv', '2021': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmp88p9g82n.csv', '2020': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv', '2019': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv', '2018': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/tmp7602cia8.csv', '2017': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/300221

In [55]:
from datetime import datetime, timedelta
now = datetime.now()
thirty_days = timedelta(days=30)
thirty_days_ago = now - thirty_days
today_datestring = now.strftime("%Y-%m-%d")
thirty_days_ago_datestring = thirty_days_ago.strftime("%Y-%m-%d")
tomorrow_datestring = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

print(today_datestring, thirty_days_ago_datestring, tomorrow_datestring)

2023-10-14 2023-09-14 2023-10-15


In [56]:
#set model folder constant
MODEL_FOLDER = './daily_models'

## Load extra features

In [57]:
#set path to mydata
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_service_id.csv'


##Define several models

In [58]:
linear_tree_model = Boston311SurvDecTree(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type','queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0,
                                      'survivalTimeFill':tomorrow_datestring},
                            files_dict=latest_URLS)

In [59]:
logistic_model = Boston311LogReg(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

In [60]:
oldlogistic_model = Boston311LogReg(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

In [61]:
logistic_tree_model = Boston311EventDecTree(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

In [102]:
kerasNLP_model = Boston311KerasNLP(train_date_range={'start':'2010-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['queue', 'subject', 'reason', 'department'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0,
                                      'survivalTimeFill':tomorrow_datestring},
                            files_dict=latest_URLS)

In [103]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

In [104]:
mydata = None

import pandas as pd
import numpy as np
import pickle

case_data_file = 'case_data.pkl'
case_data_csv = 'all_311_cases.csv'
mydata = None

X = None

if os.path.exists(case_data_file):
    mydata = pickle.load(open(case_data_file, "rb"))
else:
    data = pd.read_csv(case_data_csv)
    mydata = kerasNLP_model.load_data(data)

    pickle.dump(mydata, open(case_data_file, "wb"))




In [105]:
mydata['case_enquiry_id']

2235124    101004204966
2235125    101004204967
2235126    101004204970
2235127    101004204968
2235128    101004204972
               ...     
2689969    101005045798
2689970    101005045799
2689971    101005045800
2689972    101005045802
2689973    101005045804
Name: case_enquiry_id, Length: 454850, dtype: int64

In [106]:
mydata = kerasNLP_model.enhance_data(mydata)


In [107]:
mydata = kerasNLP_model.apply_scenario(mydata)


In [108]:

mydata = kerasNLP_model.clean_data(mydata)


In [109]:
print(mydata['case_enquiry_id'])

2235124    101004204966
2235125    101004204967
2235126    101004204970
2235127    101004204968
2235128    101004204972
               ...     
2689969    101005045798
2689970    101005045799
2689971    101005045800
2689972    101005045802
2689973    101005045804
Name: case_enquiry_id, Length: 400378, dtype: int64


In [110]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

X = None

if os.path.exists(pickle_file):
    
    X = pickle.load(open(pickle_file, "rb"))
else:
    X = pd.read_csv(EXTRA_mydata_FILE)

    #rename service_request_id to case_enquiry_id
    X.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X = X[X['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    X['cls_embedding'] = X['cls_embedding'].apply(literal_eval).apply(np.array)
    X['pooled_embedding'] = X['pooled_embedding'].apply(literal_eval).apply(np.array)

    pickle.dump(X, open(pickle_file, "wb"))



In [111]:
#print information about X2022
print(X.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265388 entries, 0 to 265387
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   case_enquiry_id   265388 non-null  int64 
 1   cls_embedding     265388 non-null  object
 2   pooled_embedding  265388 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.1+ MB
None


In [112]:
#concatenate the two dataframes and reindex
df = X

In [113]:
df.shape

(265388, 3)

In [114]:

# Assuming df is your DataFrame and it has columns 'cls_embedding' and 'pooled_embedding'
cls_embedding_flattened = np.stack(df['cls_embedding'].to_numpy())
pooled_embedding_flattened = np.stack(df['pooled_embedding'].to_numpy())

# Remove the old columns
df.drop(['cls_embedding', 'pooled_embedding'], axis=1, inplace=True)

# Add the new flattened columns
df_cls = pd.DataFrame(cls_embedding_flattened, columns=[f'cls_{i}' for i in range(cls_embedding_flattened.shape[1])])
df_pooled = pd.DataFrame(pooled_embedding_flattened, columns=[f'pooled_{i}' for i in range(pooled_embedding_flattened.shape[1])])

df = pd.concat([df, df_cls, df_pooled], axis=1)

In [115]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
is_numeric = df['case_enquiry_id'].str.isnumeric()

In [116]:
df = df[is_numeric]

In [117]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')

In [118]:
df.shape

(265388, 257)

In [119]:
df = df.drop_duplicates(subset=['case_enquiry_id']) 

In [120]:
df.shape

(265388, 257)

In [121]:
df.head()

,case_enquiry_id,cls_0,cls_1,cls_2,cls_3,cls_4,cls_5,cls_6,cls_7,cls_8,...,pooled_118,pooled_119,pooled_120,pooled_121,pooled_122,pooled_123,pooled_124,pooled_125,pooled_126,pooled_127
0,101004113559,-1.100319,0.180848,-3.067521,-2.449431,0.048062,0.750774,-1.156688,1.701071,-1.121157,...,0.090089,-0.993815,0.030800,-0.999999,-0.700883,0.967637,-0.999964,0.997570,0.920985,0.997373
1,101004113295,-0.136960,0.691521,-3.540846,-1.352687,1.299200,-0.141181,0.158119,2.410162,0.071449,...,0.257915,-0.996913,0.094944,-0.999995,-0.720977,0.951603,-0.999172,0.944971,0.856084,0.836672
2,101004113630,0.175361,0.668518,-3.556810,-1.355421,1.444425,0.603148,-1.361185,1.510217,-0.073560,...,0.026681,-0.997237,-0.056582,-0.999915,-0.639985,0.960599,-0.998327,0.998861,0.978732,0.994388
3,101004113228,-0.649289,0.929046,-2.988562,-1.767200,-0.438132,-0.361119,0.010478,1.114518,-0.448996,...,0.124731,-0.999928,0.179910,-0.999997,-0.888259,0.636486,-0.999839,0.998700,0.636364,0.968712
4,101004113229,-0.649289,0.929046,-2.988562,-1.767200,-0.438132,-0.361119,0.010478,1.114518,-0.448996,...,0.124731,-0.999928,0.179910,-0.999997,-0.888259,0.636486,-0.999839,0.998700,0.636364,0.968712


In [122]:
mydata.shape

(400378, 246)

In [123]:
mydata = mydata.drop_duplicates(subset=['case_enquiry_id'])

In [124]:
mydata.shape

(400378, 246)

In [125]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(df, on='case_enquiry_id', how='inner')


In [126]:
new_mydata.shape

(152949, 502)

In [127]:
old_bin_edges = [0, 12, 24, 72, 168, 336, 672, 1344, 2688, 9999999]
old_bin_labels = [
                "0-12 hours",      # Less than half a day
                "12-24 hours",     # Half to one day
                "1-3 days",        # One to three days
                "4-7 days",        # Four to seven days
                "1-2 weeks",       # One to two weeks
                "2-4 weeks",       # Two to four weeks
                "1-2 months",      # One to two months
                "2-4 months",      # Two to four months
                "4+ months"        # More than four months
            ]

In [128]:
bin_edges = [0, 24, 48, 72, 96, 120, 144, 168, 192, 216, 240, 264, 288, 312, 336, 360, 384, 408, 432, 456, 480, 504, 528, 552, 576, 600, 624, 648, 672, 696, 720, 744, 768, 792, 816, 840, 864, 888, 912, 936, 960, 984, 1008, 1032, 1056, 1080, 1104, 1128, 1152, 1176, 1200, 1224, 1248, 1272, 1296, 1320, 1344, 1368, 1392, 1416, 1440, 1464, 1488, 1512, 1536, 1560, 1584, 1608, 1632, 1656, 1680, 1704, 1728, 1752, 1776, 1800, 1824, 1848, 1872, 1896, 1920, 1944, 1968, 1992, 2016, 2040, 2064, 2088, 2112, 2136, 2160, 2184, 2208, 2232, 2256, 2280, 2304, 2328, 2352, 2376, 2400, 2424, 2448, 2472, 2496, 2520, 2544, 2568, 2592, 2616, 2640, 2664, 2688, 2712, 2736, 2760, 2784, 2808, 2832, 2856, 2880, 2904, 2928, 2952, 2976, 3000, 3024, 3048, 3072, 3096, 3120, 3144, 3168, 3192, 3216, 3240, 3264, 3288, 3312, 3336, 3360, 3384, 3408, 3432, 3456, 3480, 3504, 3528, 3552, 3576, 3600, 3624, 3648, 3672, 3696, 3720, 3744, 3768, 3792, 3816, 3840, 3864, 3888, 3912, 3936, 3960, 3984, 4008, 4032, 4056, 4080, 4104, 4128, 4152, 4176, 4200, 4224, 4248, 4272, 4296, 4320, 1000000]
bin_labels = [
            "0-24 hours", "1-2 days", "2-3 days", "3-4 days", "4-5 days", 
            "5-6 days", "6-7 days", "7-8 days", "8-9 days", "9-10 days",
            "10-11 days", "11-12 days", "12-13 days", "13-14 days", "14-15 days",
            "15-16 days", "16-17 days", "17-18 days", "18-19 days", "19-20 days",
            "20-21 days", "21-22 days", "22-23 days", "23-24 days", "24-25 days",
            "25-26 days", "26-27 days", "27-28 days", "28-29 days", "29-30 days",
            "30-31 days", "31-32 days", "32-33 days", "33-34 days", "34-35 days",
            "35-36 days", "36-37 days", "37-38 days", "38-39 days", "39-40 days",
            "40-41 days", "41-42 days", "42-43 days", "43-44 days", "44-45 days",
            "45-46 days", "46-47 days", "47-48 days", "48-49 days", "49-50 days",
            "50-51 days", "51-52 days", "52-53 days", "53-54 days", "54-55 days",
            "55-56 days", "56-57 days", "57-58 days", "58-59 days", "59-60 days",
            "60-61 days", "61-62 days", "62-63 days", "63-64 days", "64-65 days",
            "65-66 days", "66-67 days", "67-68 days", "68-69 days", "69-70 days",
            "70-71 days", "71-72 days", "72-73 days", "73-74 days", "74-75 days",
            "75-76 days", "76-77 days", "77-78 days", "78-79 days", "79-80 days",
            "80-81 days", "81-82 days", "82-83 days", "83-84 days", "84-85 days",
            "85-86 days", "86-87 days", "87-88 days", "88-89 days", "89-90 days",
            "90-91 days", "91-92 days", "92-93 days", "93-94 days", "94-95 days",
            "95-96 days", "96-97 days", "97-98 days", "98-99 days", "99-100 days",
            "100-101 days", "101-102 days", "102-103 days", "103-104 days", "104-105 days",
            "105-106 days", "106-107 days", "107-108 days", "108-109 days", "109-110 days",
            "110-111 days", "111-112 days", "112-113 days", "113-114 days", "114-115 days",
            "115-116 days", "116-117 days", "117-118 days", "118-119 days", "119-120 days",
            "120-121 days", "121-122 days", "122-123 days", "123-124 days", "124-125 days",
            "125-126 days", "126-127 days", "127-128 days", "128-129 days", "129-130 days",
            "130-131 days", "131-132 days", "132-133 days", "133-134 days", "134-135 days",
            "135-136 days", "136-137 days", "137-138 days", "138-139 days", "139-140 days",
            "140-141 days", "141-142 days", "142-143 days", "143-144 days", "144-145 days",
            "145-146 days", "146-147 days", "147-148 days", "148-149 days", "149-150 days",
            "150-151 days", "151-152 days", "152-153 days", "153-154 days", "154-155 days",
            "155-156 days", "156-157 days", "157-158 days", "158-159 days", "159-160 days",
            "160-161 days", "161-162 days", "162-163 days", "163-164 days", "164-165 days",
            "165-166 days", "166-167 days", "167-168 days", "168-169 days", "169-170 days",
            "170-171 days", "171-172 days", "172-173 days", "173-174 days", "174-175 days",
            "175-176 days", "176-177 days", "177-178 days", "178-179 days", "179-180 days",
            "180+ days"]
bin_number = len(bin_labels)

In [129]:
php_array = "$prediction_timespans = [\n"
for i, label in enumerate(bin_labels):
    php_array += f'    "{label}" => [{bin_edges[i]}, {bin_edges[i + 1]}],\n'
php_array += "];"

print(php_array)


$prediction_timespans = [
    "0-24 hours" => [0, 24],
    "1-2 days" => [24, 48],
    "2-3 days" => [48, 72],
    "3-4 days" => [72, 96],
    "4-5 days" => [96, 120],
    "5-6 days" => [120, 144],
    "6-7 days" => [144, 168],
    "7-8 days" => [168, 192],
    "8-9 days" => [192, 216],
    "9-10 days" => [216, 240],
    "10-11 days" => [240, 264],
    "11-12 days" => [264, 288],
    "12-13 days" => [288, 312],
    "13-14 days" => [312, 336],
    "14-15 days" => [336, 360],
    "15-16 days" => [360, 384],
    "16-17 days" => [384, 408],
    "17-18 days" => [408, 432],
    "18-19 days" => [432, 456],
    "19-20 days" => [456, 480],
    "20-21 days" => [480, 504],
    "21-22 days" => [504, 528],
    "22-23 days" => [528, 552],
    "23-24 days" => [552, 576],
    "24-25 days" => [576, 600],
    "25-26 days" => [600, 624],
    "26-27 days" => [624, 648],
    "27-28 days" => [648, 672],
    "28-29 days" => [672, 696],
    "29-30 days" => [696, 720],
    "30-31 days" => [720, 744],
    "31-3

In [130]:

df, y = kerasNLP_model.split_data(new_mydata, bin_edges=bin_edges, bin_labels=bin_labels)

In [131]:
#cast all columns that are type bool to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype('float64')

In [132]:
#list the number of rows in X and y
print(df.shape)
print(y.shape)


(152949, 499)
(152949,)


In [133]:
#best_model, best_hyperparameters = kerasNLP_model.tune_model(df, y, '/home/briarmoss/Documents/Boston_311/models/tuning')

In [134]:
#define hyperparameters
from kerastuner import HyperParameters

#set constants
start_nodes = 1024
end_nodes = 256
#l2_0 = 0.00001
#learning_rate = 7.5842e-05
l2_0 = 0.001
learning_rate = 0.0001


hp = HyperParameters()
hp.Fixed('start_nodes', start_nodes)
hp.Fixed('end_nodes', end_nodes)
hp.Fixed('l2_0', l2_0)
hp.Fixed('learning_rate', learning_rate)
hp.Fixed('final_layer', bin_number)
hp.Fixed('final_activation', 'softmax')
kerasNLP_model.best_hyperparameters = hp


#parameters for linear regression
linear='''
hp = HyperParameters()
hp.Fixed('start_nodes', start_nodes)
hp.Fixed('end_nodes', end_nodes)
hp.Fixed('l2_0', l2_0)
hp.Fixed('learning_rate', learning_rate)
hp.Fixed('final_layer', 1)
hp.Fixed('final_activation', 'linear')
kerasNLP_model.best_hyperparameters = hp
'''

In [135]:
#free all unused dataframes
df_to_delete = [cls_embedding_flattened, pooled_embedding_flattened, df_cls, df_pooled, X, new_mydata, is_numeric, mydata]

for data_frame in df_to_delete:
    del data_frame

In [136]:
import gc
gc.collect()

25

In [137]:

#parse CLS embedding column as array
test_acc = kerasNLP_model.train_model( df, y )

Starting Training at 2023-10-14 22:57:23.057345
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 1024)              512000    
                                                                 
 dense_13 (Dense)            (None, 512)               524800    
                                                                 
 dense_14 (Dense)            (None, 256)               131328    
                                                                 
 dense_15 (Dense)            (None, 181)               46517     
                                                                 
Total params: 1214645 (4.63 MB)
Trainable params: 1214645 (4.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
<class 'pandas.core.frame.DataFrame'> (122359, 181)
<class 'pandas.core.frame.DataFrame'> (3059

## Train several models

In [ ]:
print("learning is fun!") 

learning is fun!


In [ ]:
#logistic_tree_model.run_pipeline()

In [ ]:
#logistic_model.run_pipeline()

In [ ]:
import gc
gc.collect()

2290

In [ ]:
#linear_tree_model.run_pipeline()

In [ ]:
import datetime

def save_model_to_dir(model, folder_name):
    dir_path = os.path.join(MODEL_FOLDER, folder_name)
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    model_name = timestamp + "_" + model.model_type
    properties_name = model_name
    
    model.save(dir_path, model_name, properties_name)

# List of models
models = [kerasNLP_model]


# Iterate over models and save
for model in models:
    save_model_to_dir(model, model.model_type)


/home/briarmoss/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
"""
data = kerasNLP_model.load_data( 'predict' )
data = kerasNLP_model.enhance_data( data, 'predict')
clean_data = kerasNLP_model.clean_data_for_prediction( data )

X_predict, y_predict = kerasNLP_model.split_data( clean_data )
y_predict = kerasNLP_model.model.predict(X_predict)
data['survival_prediction'] = y_predict
return data
"""


"\ndata = kerasNLP_model.load_data( 'predict' )\ndata = kerasNLP_model.enhance_data( data, 'predict')\nclean_data = kerasNLP_model.clean_data_for_prediction( data )\n\nX_predict, y_predict = kerasNLP_model.split_data( clean_data )\ny_predict = kerasNLP_model.model.predict(X_predict)\ndata['survival_prediction'] = y_predict\nreturn data\n"

In [ ]:

import datetime

def save_model_to_dir(model, folder_name):
    dir_path = os.path.join(MODEL_FOLDER, folder_name)
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    model_name = timestamp + "_" + model.model_type
    properties_name = model_name
    
    model.save(dir_path, model_name, properties_name)

# List of models
models = [kerasNLP_model]


# Iterate over models and save
for model in models:
    save_model_to_dir(model, model.model_type)

